In [1]:
#SCRIPT
import os # Maneja los arcchivos de la PC
import pandas as pd # Maneja los archivos PRN
import xlwings as xw # Maneja y modifica archivos Excel con o sin macros.

def leer_prn_flexible(prn_file):
    """
    Lee un PRN detectando si la primera línea es un preámbulo tipo 'Units: ...'
    y probando distintos separadores y codificaciones.
    Devuelve (df, None) si ok o (None, 'mensaje de error') si falla.
    """
    # ¿Hay preámbulo en la primera línea no vacía?
    skiprows = 0
    try:
        with open(prn_file, 'r', encoding='utf-8', errors='ignore') as f:
            for line in f:
                if line.strip():  # primera línea no vacía
                    first = line.strip().lower().lstrip('"')
                    if first.startswith('units:'):
                        skiprows = 1
                    break
    except Exception:
        # si hay problemas leyendo como utf-8, igual seguimos a read_csv
        pass

    # Intentos de lectura (delimitador y codificación)
    intentos_sep = [
        dict(sep=None, engine='python'),   # inferencia automática
        dict(sep='\t', engine='python'),   # tab
        dict(sep=',', engine='python'),    # coma
        dict(sep=';', engine='python'),    # punto y coma
        dict(sep=r'\s+', engine='python'), # 1+ espacios
    ]
    intentos_enc = ['utf-8', 'utf-8-sig', 'latin-1']

    ultimo_error = None
    for enc in intentos_enc:
        for kw in intentos_sep:
            try:
                df = pd.read_csv(
                    prn_file,
                    header=0,
                    skiprows=skiprows,
                    encoding=enc,
                    on_bad_lines='skip',
                    **kw
                )
                # Limpieza básica de nombres de columnas
                df.columns = (
                    pd.Index(df.columns)
                    .astype(str)
                    .str.replace('\ufeff', '', regex=False)  # BOM
                    .str.strip()
                )

                # Si por error quedó una fila duplicando el header, elimínala
                if not df.empty:
                    cols = [str(c).strip().lower() for c in df.columns]
                    first_row = [str(x).strip().lower() for x in df.iloc[0].tolist()]
                    if cols == first_row:
                        df = df.iloc[1:].reset_index(drop=True)

                return df, None
            except Exception as e:
                ultimo_error = str(e)
                continue

    return None, f"No se pudo leer con separadores/codificaciones comunes. Último error: {ultimo_error}"

# =========================================
# CONFIGURACIÓN: SOLO MODIFICA 3 parámetros
# =========================================
# 1) Ruta al archivo Excel .xlsm con macros
xlsm_file = r"C:\Users\nasca\Desktop\Python Scripts\AUTOMATIZACIÓN PRN a EXCEL_FACTURACIÓN\Libres Ensa_09_2025.xlsm"
# 2) Carpeta donde se encuentran los archivos PRN
prn_folder = r"C:\Users\nasca\Desktop\Python Scripts\AUTOMATIZACIÓN PRN a EXCEL_FACTURACIÓN\PRNs a COPIAR"

# 3) Días que tiene el mes a procesar (p.ej. 31, 30, 29)
days_in_month = 30

# Cantidad de lecturas esperadas (96 registros por día)
expected_readings = days_in_month * 96

# =========================================
# FIN CONFIGURACIÓN
# =========================================

# Estructura para registrar problemas y mostrarlos en tabla al final
issues_list = [] #Cre una variable vacía para allí agregar issues 

# Inicia Excel (puedes poner visible=True si deseas depurar)
app = xw.App(visible=False)
wb = app.books.open(xlsm_file)

# Convierte las hojas en lista
all_sheets_list = list(wb.sheets)

# Omitir la primera hoja y las últimas 5
if len(all_sheets_list) <= 6:
    print("\n[ADVERTENCIA] El libro no tiene suficientes hojas para "
          "omitir la primera y las últimas 5.")
    sheets_to_process = []
else:
    sheets_to_process = all_sheets_list[1:-2]

for sheet in sheets_to_process:
    sheet_name = sheet.name
    print(f"\nProcesando hoja: '{sheet_name}'")

    # Leer el número de suministro desde la celda E2
    numero = sheet.range("E2").value
    if not numero:
        print(f"  - Celda E2 vacía. Se omite la hoja '{sheet_name}'.")
        issues_list.append({
            'Hoja': sheet_name,
            'Suministro': None,
            'Problema': 'E2 vacía (no se procesó)',
            'LecturasEncontradas': None,
            'LecturasEsperadas': expected_readings
        })
        continue

    # Convertir a string de 8 dígitos
    try:
        numero_str = str(int(numero)).zfill(8)
    except ValueError:
        print(f"  - Valor en E2 no es numérico válido: '{numero}'. Se omite.")
        issues_list.append({
            'Hoja': sheet_name,
            'Suministro': numero,
            'Problema': 'Valor E2 no numérico',
            'LecturasEncontradas': None,
            'LecturasEsperadas': expected_readings
        })
        continue

    # Construir ruta al PRN
    prn_file = os.path.join(prn_folder, f"{numero_str}.prn")
    if not os.path.exists(prn_file):
        print(f"  - No existe el archivo PRN: '{prn_file}'. Se omite.")
        issues_list.append({
            'Hoja': sheet_name,
            'Suministro': numero_str,
            'Problema': f"No se encontró {numero_str}.prn",
            'LecturasEncontradas': None,
            'LecturasEsperadas': expected_readings
        })
        continue

    # Leer el archivo PRN
    
    df_prn, read_err = leer_prn_flexible(prn_file)
    if read_err:
        print(f"   -Error al leer :c'{prn_file}': {read_err}")
        issues_list.append({
            'Hoja': sheet_name,
            'Suministro': numero_str,
            'Problema': f"Error lectura PRN: {read_err}",
            'LecturasEncontradas': None,
            'LecturasEsperadas': expected_readings
        })
        continue

    # Verificar número de filas
    total_filas = len(df_prn)
    if total_filas != expected_readings:
        print(f"  - Se esperaban {expected_readings} lecturas, hay {total_filas}. Se omite.")
        issues_list.append({
            'Hoja': sheet_name,
            'Suministro': numero_str,
            'Problema': 'Lecturas diferentes',
            'LecturasEncontradas': total_filas,
            'LecturasEsperadas': expected_readings
        })
        continue

    # Verificar columnas 'kWh-Del' y 'kVARh-Q1'
    if not {'kWh-Del', 'kVARh-Q1'}.issubset(df_prn.columns):
        print(f"  - Faltan columnas 'kWh-Del' o 'kVARh-Q1'. Se omite.")
        issues_list.append({
            'Hoja': sheet_name,
            'Suministro': numero_str,
            'Problema': "Faltan columnas 'kWh-Del' o 'kVARh-Q1'",
            'LecturasEncontradas': total_filas,
            'LecturasEsperadas': expected_readings
        })
        continue

    # Extraer listas
    kwh_del_values = df_prn['kWh-Del'].dropna().tolist()
    kvarh_q1_values = df_prn['kVARh-Q1'].dropna().tolist()

    if len(kwh_del_values) != len(kvarh_q1_values):
        print(f"  - 'kWh-Del' y 'kVARh-Q1' tienen longitudes distintas. Se omite.")
        issues_list.append({
            'Hoja': sheet_name,
            'Suministro': numero_str,
            'Problema': "Columnas kWh-Del y kVARh-Q1 con longitudes distintas",
            'LecturasEncontradas': len(df_prn),
            'LecturasEsperadas': expected_readings
        })
        continue

    # Si llega aquí, pasó todas las validaciones => Copiar datos a Excel
    print(f"  - Copiando datos en la hoja '{sheet_name}' (Suministro={numero_str}).")

    two_columns = [
        [kwh_del_values[i], kvarh_q1_values[i]]
        for i in range(len(kwh_del_values))
    ]
    start_row = 17
    start_col = 6  # Col F
    end_row = start_row + len(two_columns) - 1
    end_col = start_col + 1  # Col G

    sheet.range((start_row, start_col), (end_row, end_col)).value = two_columns
    print(f"    * Se copiaron {len(two_columns)} lecturas en columnas F y G.")

# Al finalizar, guardamos y cerramos
wb.save()
wb.close()
app.quit()
print(f"\nEl archivo Excel '{xlsm_file}' se ha actualizado.")

# ==================================================
# MOSTRAR RESUMEN DE ERRORES EN JUPYTER (SI EXISTEN)
# ==================================================
if issues_list:
    df_issues = pd.DataFrame(issues_list)
    print("\nResumen de problemas encontrados:")
    display(df_issues)  # En Jupyter, display() mostrará la tabla con formato
else:
    print("\nNo se encontraron problemas en los archivos procesados.")



Procesando hoja: 'Backus'
  - Copiando datos en la hoja 'Backus' (Suministro=35707003).
    * Se copiaron 2880 lecturas en columnas F y G.

Procesando hoja: 'NICOLL '
  - Copiando datos en la hoja 'NICOLL ' (Suministro=25820885).
    * Se copiaron 2880 lecturas en columnas F y G.

Procesando hoja: 'CINEPLANET'
  - Copiando datos en la hoja 'CINEPLANET' (Suministro=25812097).
    * Se copiaron 2880 lecturas en columnas F y G.

Procesando hoja: 'Empresa CENCOSUD-MT '
  - Copiando datos en la hoja 'Empresa CENCOSUD-MT ' (Suministro=26457805).
    * Se copiaron 2880 lecturas en columnas F y G.

Procesando hoja: 'Empresa CENCOSUD-BT'
  - Copiando datos en la hoja 'Empresa CENCOSUD-BT' (Suministro=35471447).
    * Se copiaron 2880 lecturas en columnas F y G.

Procesando hoja: 'MALL 2'
  - Copiando datos en la hoja 'MALL 2' (Suministro=25814814).
    * Se copiaron 2880 lecturas en columnas F y G.

Procesando hoja: 'REAL PLAZA-MALL 1'
  - Copiando datos en la hoja 'REAL PLAZA-MALL 1' (Suminis

,Hoja,Suministro,Problema,LecturasEncontradas,LecturasEsperadas
0,Mol_San Fernando,25650749,Lecturas diferentes,420.0,2880
1,CASTILLO_25752631,25752631,No se encontró 25752631.prn,NaN,2880
2,LA_REPUBLICA,25596626,No se encontró 25596626.prn,NaN,2880
3,ESCALY,25647298,Lecturas diferentes,2751.0,2880
4,AGRIC_LATINO_35891595,35891595,Lecturas diferentes,2274.0,2880
5,INV.MOCCE_26382295,26382295,Lecturas diferentes,1696.0,2880
6,Santa_Cruz_Lizaida,27512750,Lecturas diferentes,1680.0,2880
7,LadrilloTorreFuerte39401262,39401262,Lecturas diferentes,1699.0,2880
8,Molino del Agricultor,27512714,Lecturas diferentes,1678.0,2880
9,MOLINO'S CRISTO MORADO SAC,25765809,Lecturas diferentes,2273.0,2880


In [1]:
a=[1,2,3,4]
for i in a:
    print(a[:-1])

[1, 2, 3]
[1, 2, 3]
[1, 2, 3]
[1, 2, 3]
